## Урок 3. Парсинг html, MongoDB

### Задание 1.

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы

3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта


In [54]:
from pprint import pprint
import json
from bs4 import BeautifulSoup as bs
from requests import get
import string
from pymongo import MongoClient
from pprint import pprint

headers   = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
hh = 0
sj = 1

# список профессий для поиска
prof_list = [['Data+scientist','Слесарь'],   
             ['data-scientist','slesar']]     

# список страниц для поиска
page_list = [['0', '1'],
             ['0', '1']]     

# список сайтов для поиска
site_list = ['https://hh.ru/search/vacancy?area=1&st=searchVacancy&text=Prof_Name&page=Page_Number',
             'https://russia.superjob.ru/vakansii/Prof_Name.html?page=Page_Number'] 

# список валют
currency_list = ['руб.', 'USD', 'EUR', 'гривен']

# возвращает значение максимального и минимального размера оплаты из объявления
def find_salary(vacancy_salary):
    
    # удаляем побелы и специальные символы
    vacancy_salary = vacancy_salary.replace(u'\xa0', '')
    vacancy_salary = vacancy_salary.replace(' ', '')

    # удаляем валюту из строки и сохраняем ее наименование в переменной salary_currency
    for currency in currency_list:
        if vacancy_salary.rfind(currency)>0:
            salary_currency = currency
            vacancy_salary = vacancy_salary.replace(currency, '')
    
    # ищем минимальную и максимальную зарплату в строке   
    if vacancy_salary == "":      # значения по умолчанию, если в вакансии нет указания з\п
        salary_currency = "руб."  
        vacancy_salary_min = 0
        vacancy_salary_max = 99999999
    elif vacancy_salary.startswith('от'):
        vacancy_salary = vacancy_salary.replace('от', '')
        vacancy_salary_min = int(vacancy_salary)
        vacancy_salary_max = 99999999
    elif vacancy_salary.startswith('до'):
        vacancy_salary = vacancy_salary.replace('до', '')
        vacancy_salary_min = 0
        vacancy_salary_max = int(vacancy_salary)
    elif vacancy_salary.isdigit():
        vacancy_salary_min = int(vacancy_salary)
        vacancy_salary_max = int(vacancy_salary)
    else:
        vacancy_salary = vacancy_salary.replace('—', '-')
        vacancy_salary = vacancy_salary.split('-',1)
        vacancy_salary_min = int(vacancy_salary[0])
        vacancy_salary_max = int(vacancy_salary[1])
 
    return vacancy_salary_min, vacancy_salary_max, salary_currency


# возвращает перечень структурированных данных из переданного тега вакансии c заданного сайта
def find_vacancy(vacancy, site):
    
    # поиск и разбо информации в теге
    if site == hh:
        vacancy_name   = vacancy.find_all('div',{'class':'resume-search-item__name'})[site].getText()
        vacancy_href   = str(vacancy.find_all('a',  {'class':'bloko-link'})[site]).split()[7][5:].strip('"') 
        vacancy_site   = "hh.ru"
        vacancy_salary = vacancy.find_all('div',{'class':'vacancy-serp-item__sidebar'})[site].getText()      
    elif site == sj:
        vacancy_name   = vacancy.find('a',{'class':'icMQ_'}).getText()
        vacancy_href   = "www.superjob.ru"+str(vacancy.find('a',{'class':'icMQ_'})).split('href="')[1].split('"')[0]
        vacancy_site   = "superjob.ru"
        vacancy_salary = vacancy.find('span',{'class':'f-test-text-company-item-salary'}).getText()    
        # перевод в формат hh - пустая строка, если з\п не установлена
        if vacancy_salary == 'По договорённости':
            vacancy_salary = ""  
        # перевод в формат hh - так как replace не работает с данными символами заменяем ₽ на руб.
        currency_pos = vacancy_salary.find('₽')
        if currency_pos > 0:
            vacancy_salary = vacancy_salary[:currency_pos]+'руб.'
    else:
        vacancy_name   = "неизвестная вакансия"
        vacancy_href   = "неизвестная ссылка" 
        vacancy_site   = "неизвестный сайт"
        vacancy_salary = ""         
        
    vacancy_salary_min, vacancy_salary_max, salary_currency = find_salary(vacancy_salary)
    
    vacancy_item = {}
    vacancy_item['name'] = vacancy_name
    vacancy_item['href'] = vacancy_href
    vacancy_item['site'] = vacancy_site
    vacancy_item['salary_min'] = vacancy_salary_min
    vacancy_item['salary_max'] = vacancy_salary_max
    vacancy_item['salary_currency'] = salary_currency

    return vacancy_item

# создаем базу данных "parsing" в MongoDB
client  = MongoClient('localhost',27017)
db = client['parsing']
#vacs_hh = db.headhunter_vacancies
#vacs_sj = db.superjob_vacancies
vacs = db.parsing

# поиск на HeadHunter и запись в базу данных MongoDB "parsing"
for prof_name in prof_list[hh]:
    for page_number in page_list[hh]:
        site_name = site_list[hh].replace('Prof_Name',prof_name).replace('Page_Number',page_number)
        link = get(site_name, headers=headers)
        if link.status_code == 200:
            html = bs(link.text,'html')
            vacancy_list = html.find_all('div',{'class':'vacancy-serp-item__row_header'})
            # выбор и парсинг необходимых данных из списка ссылок
            for vacancy_message in vacancy_list:
                vacancy_item = find_vacancy(vacancy_message, hh)
                #vacancies.append(vacancy_item)
                if vacs.count_documents(vacancy_item) == 0:
                    vacs.insert_one(vacancy_item)
    
# поиск на SuperJob и запись в базу данных MongoDB "parsing"
for prof_name in prof_list[sj]:
    for page_number in page_list[sj]:
        site_name = site_list[sj].replace('Prof_Name',prof_name).replace('Page_Number',page_number)
        link = get(site_name, headers=headers)
        if link.status_code == 200:
            html = bs(link.text,'html')
            vacancy_list = html.find_all('div',{'class':'f-test-vacancy-item'})
            # выбор и парсинг необходимых данных из списка ссылок
            for vacancy_message in vacancy_list:
                vacancy_item = find_vacancy(vacancy_message, sj)
                #vacancies.append(vacancy_item)
                if vacs.count_documents(vacancy_item) == 0:
                    vacs.insert_one(vacancy_item)


# запрос желаемой зарпалты
salary = int(input('Введите желаемую зарплату: '))

# отсортировываем лишние записи без уровня з/п из базы данных, в которые мы заносили 99999999
items = vacs.find({'salary_max':{'$ne':99999999}})
for item in items:
    if item['salary_max'] >= salary:
        print('Вакансия: ', item['name'])
        print('Зарплата от: ', item['salary_min'], 'до: ', item['salary_max'],' ', item['salary_currency'])
        print('Cайт: ', item['site'], 'ссылка: ', item['href'])
        print()



Введите желаемую зарплату: 100000
Вакансия:  Junior ML Engineer / Data scientist
Зарплата от:  80000 до:  120000   руб.
Cайт:  hh.ru ссылка:  https://hh.ru/vacancy/35459810?query=Data%20scientist

Вакансия:  Middle/Senior data scientist (fraud prevention)
Зарплата от:  140000 до:  220000   руб.
Cайт:  hh.ru ссылка:  https://hh.ru/vacancy/35467008?query=Data%20scientist

Вакансия:  Data scientist
Зарплата от:  130000 до:  500000   руб.
Cайт:  hh.ru ссылка:  https://hh.ru/vacancy/35546587?query=Data%20scientist

Вакансия:  Ведущий аналитик данных / Data Scientist
Зарплата от:  0 до:  300000   руб.
Cайт:  hh.ru ссылка:  https://hh.ru/vacancy/35606282?query=Data%20scientist

Вакансия:  Data Scientist
Зарплата от:  80000 до:  180000   руб.
Cайт:  hh.ru ссылка:  https://hh.ru/vacancy/35341212?query=Data%20scientist

Вакансия:  Data Scientist (NLP)
Зарплата от:  0 до:  250000   руб.
Cайт:  hh.ru ссылка:  https://hh.ru/vacancy/33811532?query=Data%20scientist

Вакансия:  Аналитик данных / Data 